# Task 4: General Health Query Chatbot (Prompt Engineering Based)

## Objective
Create a chatbot that can answer general health-related questions using an LLM (Large Language Model) with prompt engineering techniques.

## Tools
OpenAI GPT-3.5 via API (or free open-source model like Mistral-7B-Instruct on Hugging Face)

## Problem Statement
Health-related queries are common but require careful handling. A well-designed health chatbot can provide educational information about common ailments, symptoms, and preventive measures. This task focuses on building a safe, friendly, and informative chatbot using prompt engineering. We'll implement safety filters to prevent harmful medical advice and ensure responses are appropriate and responsible.

---

## Step 1: Import Required Libraries

In [ ]:
import os
import json
import re
from typing import Optional, List, Dict
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

# For this notebook, we'll use two approaches:
# 1. Using Hugging Face Transformers with a free model
# 2. Using OpenAI API (requires API key)

# Install required packages if needed (uncomment):
# !pip install transformers torch huggingface-hub
# !pip install openai  # if using OpenAI API

## Step 2: Approach 1 - Using Hugging Face Transformers (Free, Recommended)

This approach uses an open-source model from Hugging Face that doesn't require API keys.

In [ ]:
try:
    from transformers import pipeline
    print("✅ Transformers library available!")
    TRANSFORMERS_AVAILABLE = True
except ImportError:
    print("❌ Transformers library not installed.")
    print("Install with: pip install transformers torch")
    TRANSFORMERS_AVAILABLE = False

In [ ]:
# Initialize the text generation pipeline using a free model
# Options: 'gpt2', 'distilgpt2', 'EleutherAI/gpt-neo-125m', etc.

if TRANSFORMERS_AVAILABLE:
    print("Loading model... (first run may take a few minutes)")
    
    # Using DistilGPT2 - lightweight and free
    # You can also try: 'EleutherAI/gpt-neo-125m' or 'facebook/opt-125m'
    try:
        generator = pipeline(
            'text-generation',
            model='distilgpt2',
            device=-1  # -1 for CPU, 0 for GPU if available
        )
        print("✅ Model loaded successfully!")
        MODEL_LOADED = True
    except Exception as e:
        print(f"Error loading model: {e}")
        MODEL_LOADED = False
else:
    MODEL_LOADED = False

## Step 3: Define Safety Filters and Health Categories

In [ ]:
# Define high-risk keywords that require emergency response
EMERGENCY_KEYWORDS = [
    'suicide', 'kill myself', 'harm myself', 'overdose',
    'choking', 'chest pain', 'heart attack', 'stroke',
    'severe bleeding', 'unconscious', 'unable to breathe',
    'poisoning', 'broken bone', 'severe injury'
]

# Define forbidden medical advice topics
FORBIDDEN_TOPICS = [
    'dosage', 'medication dose', 'drug dose',
    'perform surgery', 'surgery at home',
    'abortion', 'terminate pregnancy',
    'euthanasia', 'assisted suicide'
]

# Define valid health topics we can discuss
VALID_HEALTH_TOPICS = [
    'symptoms', 'causes', 'prevention', 'risk factors',
    'home remedies', 'when to see doctor', 'general information',
    'lifestyle changes', 'diet', 'exercise', 'sleep',
    'stress management', 'hygiene', 'first aid basics'
]

print("✅ Safety filters defined!")
print(f"Emergency keywords count: {len(EMERGENCY_KEYWORDS)}")
print(f"Forbidden topics count: {len(FORBIDDEN_TOPICS)}")

## Step 4: Define Prompt Engineering Templates

In [ ]:
# System prompt for the health chatbot
SYSTEM_PROMPT = """You are a friendly, helpful, and responsible medical information assistant.

IMPORTANT GUIDELINES:
1. You provide GENERAL EDUCATIONAL INFORMATION about health topics, not medical diagnoses or treatment plans.
2. Always remind users to consult a healthcare professional for personal medical decisions.
3. NEVER provide specific medication doses or emergency instructions.
4. Be empathetic and supportive but maintain professional boundaries.
5. If unsure about something, recommend consulting a doctor.
6. Focus on prevention, healthy lifestyle, and when to seek professional help.

You respond in a clear, friendly, and easy-to-understand manner."""

# Example responses for different scenarios
SAFETY_RESPONSE = """I understand you may be in distress. Please:
- Contact emergency services immediately (911 in US, 112 in Europe, 999 in UK)
- Call your local poison control center
- Reach out to a mental health crisis line
- Tell a trusted family member or friend

I'm here to provide general health information, but this situation needs immediate professional help."""

FORBIDDEN_RESPONSE = """I can't provide specific advice on this topic. This is an area where you need professional medical consultation.

Please consult with:
- Your primary care physician
- A specialist in this medical field
- A licensed pharmacist (for medications)
- A mental health professional (for psychological concerns)

I'm happy to discuss general health information on other topics!"""

print("✅ Prompt templates defined!")

## Step 5: Create Safety Filter Functions

In [ ]:
def is_emergency_query(text: str) -> bool:
    """
    Check if the query contains emergency-related keywords.
    Returns True if emergency keywords are found.
    """
    text_lower = text.lower()
    
    for keyword in EMERGENCY_KEYWORDS:
        if keyword in text_lower:
            return True
    return False

def is_forbidden_topic(text: str) -> bool:
    """
    Check if the query asks for forbidden medical advice.
    Returns True if forbidden topics are found.
    """
    text_lower = text.lower()
    
    for topic in FORBIDDEN_TOPICS:
        if topic in text_lower:
            return True
    return False

def is_health_related(text: str) -> bool:
    """
    Check if the query is health-related.
    Returns True if it contains health-related keywords.
    """
    health_keywords = [
        'health', 'disease', 'symptom', 'illness', 'pain', 'medicine',
        'doctor', 'hospital', 'treatment', 'cure', 'remedy', 'exercise',
        'diet', 'nutrition', 'sleep', 'stress', 'anxiety', 'depression',
        'cold', 'flu', 'fever', 'cough', 'headache', 'allergy',
        'blood pressure', 'diabetes', 'heart', 'infection'
    ]
    
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in health_keywords)

print("✅ Safety filter functions created!")

## Step 6: Create Health Chatbot Class

In [ ]:
class HealthChatbot:
    """
    A responsible health information chatbot with safety filters and prompt engineering.
    """
    
    def __init__(self, use_transformer=True):
        """
        Initialize the chatbot.
        
        Args:
            use_transformer (bool): Whether to use Hugging Face Transformers
        """
        self.use_transformer = use_transformer and MODEL_LOADED
        self.conversation_history = []
        self.total_queries = 0
        self.blocked_queries = 0
        
        if self.use_transformer:
            self.generator = generator
    
    def check_safety(self, query: str) -> Dict[str, any]:
        """
        Check query for safety issues.
        
        Returns:
            dict with keys: 'is_safe', 'reason', 'response'
        """
        # Check for emergency
        if is_emergency_query(query):
            return {
                'is_safe': False,
                'reason': 'EMERGENCY_DETECTED',
                'response': SAFETY_RESPONSE,
                'severity': 'CRITICAL'
            }
        
        # Check for forbidden topics
        if is_forbidden_topic(query):
            return {
                'is_safe': False,
                'reason': 'FORBIDDEN_TOPIC',
                'response': FORBIDDEN_RESPONSE,
                'severity': 'HIGH'
            }
        
        # Check if it's health-related
        if not is_health_related(query):
            return {
                'is_safe': True,
                'reason': 'NON_HEALTH_QUERY',
                'response': None,
                'severity': 'NONE'
            }
        
        return {
            'is_safe': True,
            'reason': 'SAFE_QUERY',
            'response': None,
            'severity': 'NONE'
        }
    
    def generate_response(self, query: str, use_model=True) -> str:
        """
        Generate a response using either a model or template.
        
        Args:
            query (str): The user's health query
            use_model (bool): Whether to use the LLM model
        
        Returns:
            str: The chatbot's response
        """
        # Check safety first
        safety_check = self.check_safety(query)
        
        # If not safe, return safety response
        if not safety_check['is_safe']:
            self.blocked_queries += 1
            return safety_check['response']
        
        # If not health-related, respond appropriately
        if safety_check['reason'] == 'NON_HEALTH_QUERY':
            return (
                "I'm specifically designed to help with health-related questions. "
                "Your question doesn't seem to be health-related. "
                "Feel free to ask me about symptoms, prevention, healthy habits, or when to see a doctor!"
            )
        
        # Use model if available and requested
        if use_model and self.use_transformer:
            try:
                prompt = f"{SYSTEM_PROMPT}\n\nUser: {query}\n\nAssistant:"
                
                response = self.generator(
                    prompt,
                    max_length=200,
                    num_return_sequences=1,
                    do_sample=True,
                    top_p=0.9,
                    temperature=0.7,
                )
                
                # Extract response text
                full_text = response[0]['generated_text']
                generated_response = full_text.split('Assistant:')[-1].strip()
                
                return generated_response if generated_response else self._template_response(query)
            except Exception as e:
                print(f"Model error: {e}")
                return self._template_response(query)
        else:
            # Use template-based response
            return self._template_response(query)
    
    def _template_response(self, query: str) -> str:
        """
        Generate a template-based response for common health queries.
        
        Args:
            query (str): The user's query
        
        Returns:
            str: A helpful template response
        """
        query_lower = query.lower()
        
        # Common health conditions
        if 'cold' in query_lower or 'cough' in query_lower:
            return (
                "Common cold is usually caused by viruses and resolves on its own. "
                "To help manage symptoms:\n"
                "• Rest and get plenty of sleep\n"
                "• Stay hydrated - drink water, tea, or broth\n"
                "• Use a humidifier to ease congestion\n"
                "• Gargle with salt water for sore throat\n"
                "• Use saline nasal drops for congestion\n\n"
                "Seek medical attention if symptoms persist beyond 10 days or worsen significantly."
            )
        
        elif 'headache' in query_lower:
            return (
                "Headaches can have various causes. General tips to manage them:\n"
                "• Rest in a quiet, dark room\n"
                "• Apply a cold or warm compress\n"
                "• Stay hydrated\n"
                "• Practice relaxation techniques\n"
                "• Avoid triggers like stress or certain foods\n\n"
                "If headaches are frequent or severe, consult a healthcare provider."
            )
        
        elif 'fever' in query_lower:
            return (
                "Fever is often the body's way of fighting infection. General care:\n"
                "• Rest and drink plenty of fluids\n"
                "• Keep the room cool and comfortable\n"
                "• Wear light clothing\n"
                "• Monitor your temperature\n\n"
                "Seek immediate medical attention if fever exceeds 103°F (39.4°C) or lasts more than 3 days."
            )
        
        elif 'sleep' in query_lower or 'insomnia' in query_lower:
            return (
                "Good sleep hygiene tips:\n"
                "• Maintain a consistent sleep schedule\n"
                "• Keep your bedroom cool, dark, and quiet\n"
                "• Avoid screens 30-60 minutes before bed\n"
                "• Limit caffeine, especially in the afternoon\n"
                "• Exercise regularly, but not close to bedtime\n"
                "• Try relaxation techniques like deep breathing\n\n"
                "If sleep problems persist, consult a sleep specialist."
            )
        
        elif 'stress' in query_lower or 'anxiety' in query_lower:
            return (
                "Ways to manage stress and anxiety:\n"
                "• Practice deep breathing exercises\n"
                "• Engage in regular exercise\n"
                "• Try meditation or mindfulness\n"
                "• Maintain social connections\n"
                "• Limit caffeine and alcohol\n"
                "• Ensure adequate sleep\n"
                "• Consider professional mental health support\n\n"
                "If anxiety significantly impacts daily life, please seek help from a mental health professional."
            )
        
        elif 'diet' in query_lower or 'nutrition' in query_lower:
            return (
                "Healthy eating guidelines:\n"
                "• Eat a variety of fruits and vegetables\n"
                "• Choose whole grains over refined grains\n"
                "• Include lean proteins in your diet\n"
                "• Limit sugar and processed foods\n"
                "• Stay hydrated with water\n"
                "• Eat in moderation\n\n"
                "For personalized nutrition advice, consult a registered dietitian."
            )
        
        elif 'exercise' in query_lower or 'workout' in query_lower:
            return (
                "Exercise recommendations:\n"
                "• Aim for at least 150 minutes of moderate aerobic activity per week\n"
                "• Include strength training 2-3 times per week\n"
                "• Start gradually if you're new to exercise\n"
                "• Choose activities you enjoy\n"
                "• Warm up before and cool down after exercise\n"
                "• Stay hydrated during exercise\n\n"
                "Consult your doctor before starting a new exercise program, especially if you have health concerns."
            )
        
        elif 'when' in query_lower and 'doctor' in query_lower:
            return (
                "You should see a doctor if you experience:\n"
                "• Symptoms that persist longer than expected\n"
                "• Severe pain or discomfort\n"
                "• Difficulty breathing or chest pain\n"
                "• Significant changes in your health\n"
                "• Concerns about your medications\n"
                "• Mental health concerns\n\n"
                "Don't delay seeking professional help if you're unsure - it's better to be cautious with your health!"
            )
        
        else:
            return (
                f"Thanks for your question about health. While I don't have a specific answer to '{query}', "
                "I recommend:\n"
                "• Consulting with a healthcare professional\n"
                "• Checking reliable health websites (WHO, CDC, Mayo Clinic)\n"
                "• Keeping track of your symptoms\n\n"
                "I'm here to provide general health information. Feel free to ask about symptoms, prevention, "
                "lifestyle changes, or when to see a doctor!"
            )
    
    def chat(self, query: str, use_model=True) -> str:
        """
        Main chat interface.
        
        Args:
            query (str): User's question
            use_model (bool): Whether to use LLM
        
        Returns:
            str: Chatbot's response
        """
        self.total_queries += 1
        response = self.generate_response(query, use_model)
        
        # Store in history
        self.conversation_history.append({
            'user': query,
            'bot': response,
            'timestamp': len(self.conversation_history)
        })
        
        return response
    
    def get_statistics(self) -> Dict:
        """
        Get chatbot usage statistics.
        """
        return {
            'total_queries': self.total_queries,
            'blocked_queries': self.blocked_queries,
            'allowed_queries': self.total_queries - self.blocked_queries,
            'block_rate': f"{(self.blocked_queries / self.total_queries * 100) if self.total_queries > 0 else 0:.1f}%"
        }

print("✅ HealthChatbot class created!")

## Step 7: Initialize the Chatbot

In [ ]:
# Initialize the chatbot
# Set use_model=True to use Hugging Face Transformers (if available)
# Set use_model=False to use template-based responses

chatbot = HealthChatbot(use_transformer=True)

print("✅ Chatbot initialized!")
print(f"Model loaded: {chatbot.use_transformer}")
if chatbot.use_transformer:
    print("Using Hugging Face Transformers for responses")
else:
    print("Using template-based responses (no model loaded)")

## Step 8: Test with Example Queries - Safe Queries

In [ ]:
# Test with safe, health-related queries
safe_queries = [
    "What are the common symptoms of a cold?",
    "How can I improve my sleep quality?",
    "What are some good exercises for beginners?",
    "How can I manage stress and anxiety?",
    "What should I eat to maintain a healthy diet?"
]

print("\n" + "="*70)
print("TESTING SAFE HEALTH QUERIES")
print("="*70)

for query in safe_queries[:3]:  # Test first 3 to save time
    print(f"\n👤 User: {query}")
    response = chatbot.chat(query, use_model=False)  # Use template responses for speed
    print(f"🤖 Bot: {response}")
    print("-" * 70)

## Step 9: Test with Example Queries - Emergency Cases

In [ ]:
# Test with emergency queries (should be blocked)
emergency_queries = [
    "I'm having severe chest pain",
    "I want to harm myself",
    "I took an overdose of medicine"
]

print("\n" + "="*70)
print("TESTING EMERGENCY QUERIES (BLOCKED)")
print("="*70)

for query in emergency_queries[:2]:  # Test first 2
    print(f"\n👤 User: {query}")
    response = chatbot.chat(query, use_model=False)
    print(f"🤖 Bot: {response}")
    print("-" * 70)
    print("⚠️ [EMERGENCY DETECTED - SAFETY FILTER ACTIVATED]")
    print("-" * 70)

## Step 10: Test with Example Queries - Forbidden Topics

In [ ]:
# Test with forbidden topic queries (should be blocked)
forbidden_queries = [
    "What is the correct dosage for aspirin?",
    "How can I perform surgery at home?"
]

print("\n" + "="*70)
print("TESTING FORBIDDEN TOPIC QUERIES (BLOCKED)")
print("="*70)

for query in forbidden_queries:
    print(f"\n👤 User: {query}")
    response = chatbot.chat(query, use_model=False)
    print(f"🤖 Bot: {response}")
    print("-" * 70)
    print("🛑 [FORBIDDEN TOPIC - SAFETY FILTER ACTIVATED]")
    print("-" * 70)

## Step 11: Test with Non-Health Queries

In [ ]:
# Test with non-health queries
non_health_queries = [
    "What is the weather like today?",
    "Tell me a joke",
    "How do I cook pasta?"
]

print("\n" + "="*70)
print("TESTING NON-HEALTH QUERIES")
print("="*70)

for query in non_health_queries[:1]:  # Test first 1
    print(f"\n👤 User: {query}")
    response = chatbot.chat(query, use_model=False)
    print(f"🤖 Bot: {response}")
    print("-" * 70)

## Step 12: Interactive Chatbot Session

In [ ]:
def run_chatbot_session(num_queries=5):
    """
    Run an interactive chatbot session for testing.
    
    Args:
        num_queries (int): Number of sample queries to run
    """
    print("\n" + "="*70)
    print("INTERACTIVE CHATBOT SESSION")
    print("="*70)
    
    sample_queries = [
        "I have a persistent headache, what should I do?",
        "How can I boost my immune system?",
        "What are the signs I should see a doctor?",
        "How much water should I drink daily?",
        "What are good exercises for weight loss?"
    ]
    
    for i, query in enumerate(sample_queries[:num_queries], 1):
        print(f"\n[Query {i}/{num_queries}]")
        print(f"👤 User: {query}")
        response = chatbot.chat(query, use_model=False)
        print(f"🤖 Bot: {response}")
        print("-" * 70)

# Run the session
run_chatbot_session(num_queries=3)

## Step 13: Display Chatbot Statistics

In [ ]:
# Get and display statistics
stats = chatbot.get_statistics()

print("\n" + "="*70)
print("CHATBOT STATISTICS")
print("="*70)
print(f"Total Queries Processed: {stats['total_queries']}")
print(f"Allowed Queries: {stats['allowed_queries']}")
print(f"Blocked Queries (Safety): {stats['blocked_queries']}")
print(f"Block Rate: {stats['block_rate']}")
print("="*70)

## Step 14: Display Conversation History

In [ ]:
# Display conversation history
print("\n" + "="*70)
print("CONVERSATION HISTORY")
print("="*70)

if len(chatbot.conversation_history) > 0:
    for i, exchange in enumerate(chatbot.conversation_history, 1):
        print(f"\nExchange {i}:")
        print(f"User: {exchange['user']}")
        print(f"Bot: {exchange['bot'][:100]}..." if len(exchange['bot']) > 100 else f"Bot: {exchange['bot']}")
        print("-" * 70)
else:
    print("No conversation history available yet.")

## Step 15: Key Design Decisions and Best Practices

In [ ]:
design_decisions = """
KEY DESIGN DECISIONS & BEST PRACTICES:

1. SAFETY FILTERS:
   - Emergency Detection: Identifies critical keywords and provides crisis resources
   - Forbidden Topics: Prevents giving specific medical advice on restricted topics
   - Health Relevance: Only responds to health-related queries

2. PROMPT ENGINEERING:
   - Clear System Prompt: Defines the chatbot's role and limitations
   - Guidelines: Ensures responses are educational, not prescriptive
   - Empathy: Maintains professional but warm tone

3. DUAL APPROACH:
   - Model-Based (LLM): Uses Hugging Face Transformers for more natural responses
   - Template-Based: Provides reliable responses for common conditions
   - Fallback: Uses templates if model fails or is unavailable

4. RESPONSE QUALITY:
   - Educational Focus: Teaches prevention and healthy habits
   - Professional Referral: Always recommends consulting healthcare providers
   - Structured Information: Uses bullet points for clarity
   - Appropriate Length: Keeps responses concise but comprehensive

5. RESPONSIBLE AI:
   - Disclaimers: Clear that bot cannot replace medical professionals
   - Monitoring: Tracks blocked queries to identify edge cases
   - Transparency: Explains why certain queries can't be answered
   - Limitations: Honest about what the bot can and cannot do

6. USER EXPERIENCE:
   - Conversation History: Tracks exchanges for quality assurance
   - Statistics: Monitors safety filter effectiveness
   - Friendly Tone: Uses emojis and clear formatting
   - Error Handling: Gracefully handles model failures
"""

print(design_decisions)

## Step 16: Deployment Recommendations

In [ ]:
deployment_guide = """
DEPLOYMENT RECOMMENDATIONS:

1. ENVIRONMENT SETUP:
   pip install transformers torch huggingface-hub
   # Optional for OpenAI API:
   pip install openai

2. API INTEGRATION OPTIONS:
   
   Option A: Hugging Face Inference API
   - Free tier available
   - No local GPU required
   - Good for low-traffic applications
   
   Option B: Local Model (Current Implementation)
   - Privacy-friendly (data stays local)
   - No API costs
   - Requires computational resources
   
   Option C: OpenAI API
   - Most powerful models
   - Requires API key and payment
   - Best quality responses

3. FRONTEND FRAMEWORKS:
   - Streamlit: Quick web interface (pip install streamlit)
   - Flask/FastAPI: RESTful API endpoints
   - Telegram Bot: Messaging app integration
   - Discord Bot: Community platform integration

4. MONITORING & LOGGING:
   - Log all queries for audit trail
   - Monitor false negatives (missed emergencies)
   - Track user satisfaction metrics
   - Alert on emergency keywords

5. COMPLIANCE:
   - Add clear medical disclaimer
   - HIPAA compliance if in healthcare
   - GDPR compliance for EU users
   - Regular security audits

6. SAMPLE STREAMLIT CODE:
   ```python
   import streamlit as st
   from health_chatbot import HealthChatbot
   
   st.title("Health Information Chatbot")
   
   if 'chatbot' not in st.session_state:
       st.session_state.chatbot = HealthChatbot()
   
   user_input = st.text_input("Ask a health question:")
   if user_input:
       response = st.session_state.chatbot.chat(user_input)
       st.write(response)
   ```
"""

print(deployment_guide)

## Summary

In this task, we successfully:
1. ✅ Created a health information chatbot using prompt engineering
2. ✅ Implemented comprehensive safety filters for emergencies
3. ✅ Built forbidden topic detection to prevent harmful advice
4. ✅ Integrated Hugging Face Transformers for natural responses
5. ✅ Created template-based responses for common conditions
6. ✅ Tested the chatbot with safe, emergency, and forbidden queries
7. ✅ Tracked conversation history and statistics
8. ✅ Implemented responsible AI practices
9. ✅ Provided deployment recommendations

**Skills Demonstrated:**
- Prompt engineering and LLM integration
- Safety filter implementation
- Natural language processing
- Conversational AI design
- Responsible AI practices
- Building production-ready chatbots
- Query classification and handling
- User experience design for AI systems
- Error handling and fallback strategies